In [167]:
import sys, os
sys.path.append(os.pardir) # 为了导入父目录中的文件而进行的设定
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
# normalize是否正规化为0.0~1.0之间的数值，如果为false则输出0～255。
# 输出各个数据的形状
print(x_train.shape) # (60000, 784)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [168]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from PIL import Image
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(type((x_train, t_train)))
print(label) # 5
print(img.shape) # (784,)
img = img.reshape(28, 28) # 把图像的形状变成原来的尺寸 print(img.shape) # (28, 28)
#img_show(img)


<class 'tuple'>
5
(784,)


In [172]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True,one_hot_label=False)
    return x_test, t_test
def init_network():
    with open("../ch03/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        return network
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    return y
x,t = get_data()
network = init_network()
print('network.type:'+str(type(network)))
for k in network:
    print(k)
accuracy_cnt = 0
print('x的类型：'+str(type(x))+'，长度:'+str(len(x))+',x[0]类型:'+str(type(x[0]))+',x[0]长度:'+str(len(x[0]))+',x[0]形状:'+str(x[0].shape))
print('t的类型：'+str(type(t))+'，长度'+str(len(t))+',t[0]类型:'+str(type(t[0])))
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 获取概率最高的元素的索引
    if p == t[i]:
        accuracy_cnt += 1
print("Accuracy:" + str(float(accuracy_cnt) / len(x_test)))

network.type:<class 'dict'>
b2
W1
b1
W2
W3
b3
x的类型：<class 'numpy.ndarray'>，长度:10000,x[0]类型:<class 'numpy.ndarray'>,x[0]长度:784,x[0]形状:(784,)
t的类型：<class 'numpy.ndarray'>，长度10000,t[0]类型:<class 'numpy.uint8'>
Accuracy:0.9352


### MNIST数据集
load_mnist()
- normalize是否正规化为0.0~1.0之间的数值，如果为false则输出0～255。
- flatten是否展开输入图像，如果设置为false，输出1x28x28的三维数组
- one_hot_lable one-hot 表示是仅正确解标签为 1，其余 皆为 0 的数组，就像 [0,0,1,0,0,0,0,0,0,0]

x的类型：<class 'numpy.ndarray'>，长度:10000,x[0]类型:<class 'numpy.ndarray'>,x[0]长度:784,x[0]形状:(784,)
t的类型：<class 'numpy.ndarray'>，长度10000,t[0]类型:<class 'numpy.uint8'>
network.type:<class 'dict'>
### 神经网络的推理处理
神经网络输入层有783个神经元，输出层有10个神经元。
其中有两个隐藏层，神经元数量可以设置为任何值。
get_data()获取数据
init_network()获取权重，偏置参数
predict()预测计算
predict()函数以NumPy数组的形式输出各个标签对应的概率.
np.argmax()函数取出数组中最大值的索引
比较神经网络所预测的答案和正确解标签,将正确概率作为识别精度.
对神经网络的输入数据 进行某种既定的转换称为预处理(pre-processing)。
预处理在神经网络(深度学习)中非常实用，其有效性已在提高识别 性能和学习的效率等众多实验中得到证明。
作为一种预处理,我们将各个像素值除以 255，进行了简单的正规化。
实际上，很多预处理都会考虑到数据的整体分布。
比如，利用数据整体的<font color=#FF0000>**均值**</font>或<font color=#FF0000>**标准差**</font>，<font color=#FF0000>**移动数据**</font>，使<font color=#FF0000>**数据整体以 0 为中心分布**</font>，
或者进行正规化，把数据的延展控制在一定范围内。
将数据整体的分布形状均匀化的方法，即数据白化(whitening)等。
### 批处理

查看输入数据和权重参数的形状



In [170]:
W1,W2,W3 = network['W1'],network['W2'],network['W3']
print('x:'+str(x.shape))
print('x[0]:'+str(x[0].shape))
print('W1:'+str(W1.shape))
print('W2:'+str(W2.shape))
print('W3:'+str(W3.shape))

x:(10000, 784)
x[0]:(784,)
W1:(784, 50)
W2:(50, 100)
W3:(100, 10)


In [171]:
batch_size = 100 # 步长，批大小
accuracy_cnt = 0
print( str(x[0:100].shape))
for i in range(0,len(x),batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network,x_batch)
    p = np.argmax(y_batch,axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

(100, 784)
Accuracy:0.9352


range(start,end,step) 开始,结束,步长.
x[i:i+batch_size],从输入的数据中抽出批数据
从第i到i+batch_n个之间的数据。x[0:100]、x[100:200]。
argmax()获取值最大元素的索引。参数axis=1。axis可取0或者1。
沿着1维的方向找到最大元素的索引.
使用NumPy数组(==)生成True、False构成的布尔数组.
使用np.sum()方法计算数组中true的个数.


## <center>小结
- 神经网络中激活函数使用平滑变化的sigmoid函数或者ReLU函数
- 可以通过NumPy多维数组,实现高效的神经网络
- 机器学习问题分为回归问题和分类问题
- 输出层激活函数,回归问题一般使用恒等函数,分类问题中一般使用softmax函数
- 分类问题中,输出层的神经元数量设置要为分类的类别数.
- 输入数据的集合称为批.通过以批为单位进行推理处理,能够实现高速运算.